In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, count, when, date_format

# Create a Spark session
spark = SparkSession.builder.appName("Transactions").getOrCreate()

# Example data, you can replace this with your actual data
data = [
    (1, "USA", "approved", 100, "2024-01-15"),
    (2, "USA", "declined", 200, "2024-01-20"),
    (3, "Canada", "approved", 150, "2024-01-25"),
    (4, "Canada", "approved", 250, "2024-02-01"),
    (5, "USA", "declined", 300, "2024-02-15")
]

# Define schema
schema = ["id", "country", "state", "amount", "trans_date"]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Convert trans_date to date type
df = df.withColumn("trans_date", col("trans_date").cast("date"))

# Extract year and month from trans_date
df = df.withColumn("year_month", date_format(col("trans_date"), "yyyy-MM"))

# Group by year_month and country and aggregate the required metrics
result_df = df.groupBy("year_month", "country").agg(
    count("id").alias("num_transactions"),
    _sum("amount").alias("total_amount"),
    count(when(col("state") == "approved", True)).alias("num_approved"),
    _sum(when(col("state") == "approved", col("amount"))).alias("approved_amount")
)

# Show the result
result_df.show()


24/09/03 10:17:20 WARN Utils: Your hostname, Kulyashs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.204 instead (on interface en0)
24/09/03 10:17:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/03 10:17:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/03 10:17:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+----------+-------+----------------+------------+------------+---------------+
|year_month|country|num_transactions|total_amount|num_approved|approved_amount|
+----------+-------+----------------+------------+------------+---------------+
|   2024-01|    USA|               2|         300|           1|            100|
|   2024-01| Canada|               1|         150|           1|            150|
|   2024-02| Canada|               1|         250|           1|            250|
|   2024-02|    USA|               1|         300|           0|           NULL|
+----------+-------+----------------+------------+------------+---------------+



24/09/03 10:17:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54640)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.8/3.8.19/Frameworks/Python.framework/Versions/3.8/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.8/3.8.19/Frameworks/Python.framework/Versions/3.8/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.8/3.8.19/Frameworks/Python.framework/Versions/3.8/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass

In [7]:
result_df.toPandas()

,year_month,country,num_transactions,total_amount,num_approved,approved_amount
0,2024-01,USA,2,300,1,100.0
1,2024-01,Canada,1,150,1,150.0
2,2024-02,Canada,1,250,1,250.0
3,2024-02,USA,1,300,0,NaN


24/09/03 11:28:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 732372 ms exceeds timeout 120000 ms
24/09/03 11:28:08 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/03 11:28:11 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$